In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import gym

In [2]:
from JSAnimation.IPython_display import display_animation
from matplotlib import animation
from IPython.display import display

def display_frames_as_gif(frames):
    plt.figure(figsize=(frames[0].shape[1]/72.0, frames[0].shape[0]/72.0), dpi=72)
    patch = plt.imshow(frames[0])
    plt.axis('off')
    
    def animate(i):
        patch.set_data(frames[i])
    
    anim = animation.FuncAnimation(plt.gcf(), animate, frames=len(frames), interval=50)
    
    anim.save('movie_cartpole_DQN.mp4')
    display(display_animation(anim, default_mode='loop'))

In [3]:
from collections import namedtuple

Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))

In [4]:
ENV = 'CartPole-v0'
GAMMA = 0.99
MAX_STEPS = 200
NUM_EPISODES = 500

In [5]:
# 経験を保存するメモリクラスを定義

class ReplayMemory:
    def __init__(self, CAPACITY):
        self.capacity = CAPACITY  # メモリの最大長
        self.memory = []                 # 経験を保存する変数
        self.index = 0                       # 保存するindexを示す変数
        
    def push(self, state, action, state_next, reward):
        ''' trainsition = (state, action, state_next, reward) をメモリに保存する　'''
        
        if len(self.memory) < self.capacity:
            self.memory.append(None)    # メモリが満タンでないときは追加
        
        # namedtuple の Transition を使用し, 値とフィールド名をペアにして保存する
        self.memory[self.index] = Transition(state, action, state_next, reward)
        self.index = (self.index + 1)%self.capacity   # 保存するindexを1つずらす
        
    def sample(self, batch_size):
        ''' batch_size 分だけ, ランダムに保存内容を取り出す '''
        return random.sample(self.memory, batch_size)
    
    def __len__(self):
        ''' 関数 len に対して, 現在の変数 memory の長さを返す '''
        return len(self.memory)

In [6]:
# エージェントが持つ脳となるクラス
# Q関数をディープラーニングのネットワークをクラスとして定義

import random
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F

BATCH_SIZE = 32
CAPACITY = 10000

class Net(nn.Module):
    def __init__(self, n_in, n_mid, n_out):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(n_in, n_mid)
        self.fc2 = nn.Linear(n_mid, n_mid)
        # Dueling network 
        self.fc3_adv = nn.Linear(n_mid, n_out)
        self.fc3_v = nn.Linear(n_mid, 1)
        
    def forward(self, x):
        h1 = F.relu(self.fc1(x))
        h2 = F.relu(self.fc2(h1))
        
        adv = self.fc3_adv(h2)      # ReLU しない
        val = self.fc3_v(h2).expand(-1, adv.size(1))  # ReLU しない
        
        output = val + adv - adv.mean(1, keepdim=True).expand(-1, adv.size(1))
        return output

class Brain:
    def __init__(self, num_states, num_actions):
        self.num_actions = num_actions        # CartPole の行動数 (右 or 左) の2を取得
        
        # 経験を記憶するメモリオブジェクトを生成
        self.memory = ReplayMemory(CAPACITY)
        
        # ニューラルネットワークを構築
        n_in, n_mid, n_out = num_states, 32, num_actions
        self.main_q_network = Net(n_in, n_mid, n_out)
        self.target_q_network = Net(n_in, n_mid, n_out)
        print(self.main_q_network)
        
        # 最適化手法の設定
        self.optimizer = optim.Adam(self.main_q_network.parameters(), lr=0.0001)
        
    def replay(self):
        ''' Experience Replay でネットワークの結合パラメータを学習 '''
        
        # 1. メモリサイズの確認
        if len(self.memory) < BATCH_SIZE:
            return
        
        # 2. ミニバッチの作成
        self.batch, self.state_batch, self.action_batch, self.reward_batch, self.non_final_next_states = self.make_minibatch()
        
        # 3. 教師信号となる Q(s_t, a_t) 値 を求める
        self.expected_state_action_values = self.get_expected_state_action_values()
        
        # 4. 結合パラメータの更新
        self.update_main_q_network()
        
    def decide_action(self, state, episode):
        ''' 現在の状態に応じて, 行動を決定する '''
        # epsilon-greedy 法で徐々に最適行動のみを採用する
        epsilon = 0.5 * (1 / (episode + 1))
        
        if epsilon <= np.random.uniform(0, 1):
            self.main_q_network.eval()
            with torch.no_grad():
                action = self.main_q_network(state).max(1)[1].view(1, 1)
            # ネットワークの出力の最大値の index を取り出す = max(1)[1]
            # .view(1, 1) は [torch.LongTensor of size 1] を 1x1　に変換
        else:
            action = torch.LongTensor([[random.randrange(self.num_actions)]])
        
        return action
    
    def make_minibatch(self):
        ''' ミニバッチの作成 '''
        # 1. メモリからミニバッチ分のデータを取り出す
        transitions = self.memory.sample(BATCH_SIZE)
        
        # 2. 各変数をミニバッチに対応する形に変更
        batch = Transition(*zip(*transitions))
        
        # 3. 各変数の要素をミニバッチに対応する形に変形する
        state_batch = torch.cat(batch.state)
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)
        non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])
        
        return batch, state_batch, action_batch, reward_batch, non_final_next_states

    def get_expected_state_action_values(self):
        ''' 教師信号となる Q(s_t, a_t) 値を求める '''
        
        # 1. ネットワークを推論モードに切り替える
        self.main_q_network.eval()
        self.target_q_network.eval()
        
        # 2. ネットワークが出力した　Q(s_t, a_t) を求める
        self.state_action_values = self.main_q_network(self.state_batch).gather(1, self.action_batch)
        
        # 3. max{Q(s_t+1, a)} 値を求める. 次の状態が存在するかに注意する.
        non_final_mask = torch.ByteTensor(tuple(map(lambda s: s is not None, self.batch.next_state)))
        next_state_values = torch.zeros(BATCH_SIZE)
        a_m = torch.zeros(BATCH_SIZE).type(torch.LongTensor)
        a_m[non_final_mask] = self.main_q_network(self.non_final_next_states).detach().max(1)[1]
        a_m_non_final_next_states = a_m[non_final_mask].view(-1, 1)
        next_state_values[non_final_mask] = self.target_q_network(self.non_final_next_states).gather(1, a_m_non_final_next_states).detach().squeeze()
        
        # 4. 教師となる　Q(s_t, a_t) 値を, Q学習の式から求める
        expected_state_action_values = self.reward_batch + GAMMA * next_state_values
        
        return expected_state_action_values
    
    def update_main_q_network(self):
        ''' 結合パラメータの更新 '''
        
        # 1. ネットワークを訓練モードに切り替える
        self.main_q_network.train()
        
        # 2. 損失感数を計算する smooth_l1_loss = Huber loss
        loss = F.smooth_l1_loss(self.state_action_values, self.expected_state_action_values.unsqueeze(1))
        
        # 3. 結合パラメータを更新する
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
    
    def update_target_q_network(self):
        ''' Target Q Network を Main と同じにする '''
        self.target_q_network.load_state_dict(self.main_q_network.state_dict())

In [7]:
class Agent:
    def __init__(self, num_states, num_actions):
        ''' 課題の状態と行動の数を設定する '''
        self.brain = Brain(num_states, num_actions)  # エージェントが行動を決定するための頭脳を生成
        
    def update_q_function(self):
        ''' Q関数を更新する '''
        self.brain.replay()
        
    def get_action(self, state, episode):
        ''' 行動を決定する '''
        action = self.brain.decide_action(state, episode)
        return action
    
    def memorize(self, state, action, state_next, reward):
        ''' memory オブジェクトに, state, action, state_next, reward の内容を保存する '''
        self.brain.memory.push(state, action, state_next, reward)
        
    def update_target_q_function(self):
        ''' Target Q-Network　を Main Q-Network と同じに更新 '''
        self.brain.update_target_q_network()

In [8]:
class Environment:
    def __init__(self):
        self.env = gym.make(ENV)
        self.num_states = self.env.observation_space.shape[0]
        self.num_actions = self.env.action_space.n
        self.agent = Agent(self.num_states, self.num_actions)
    
    def run(self):
        ''' 実行 '''
        episode_10_list = np.zeros(10)
        complete_episodes = 0
        episode_final = False
        frames = []
        
        for episode in range(NUM_EPISODES):
            observation = self.env.reset()
            
            state = observation
            state = torch.from_numpy(state).type(torch.FloatTensor)
            state = torch.unsqueeze(state, 0)
        
            for step in range(MAX_STEPS):
#                 if episode_final is True:
#                     frames.append(self.env.render(mode='rgb_array'))
                    
                action = self.agent.get_action(state, episode)
                observation_next, _, done, _ = self.env.step(action.item())
                
                if done:
                    state_next = None
                    episode_10_list = np.hstack((episode_10_list[1:], step+1))
                    
                    if step < 195:
                        reward = torch.FloatTensor([-1.0])
                        complete_episodes = 0
                    else:
                        reward = torch.FloatTensor([1.0])
                        complete_episodes = complete_episodes + 1
                else:
                    reward = torch.FloatTensor([0.0])
                    state_next = observation_next
                    state_next = torch.from_numpy(state_next).type(torch.FloatTensor)
                    state_next = torch.unsqueeze(state_next, 0)
                
                self.agent.memorize(state, action, state_next, reward)
                self.agent.update_q_function()
                state = state_next
                
                if done:
                    print('%d Episode: Finished after %d steps : 10試行の平均step数 = %.1lf' % (episode, step+1, episode_10_list.mean()))
                    if (episode %2 == 0):
                        self.agent.update_target_q_function()
                    break
            
            if episode_final is True:
                self.env.close()
#                 display_frames_as_gif(frames)
                break
            
            if complete_episodes >= 10:
                print('10回連続成功')
                episode_final = True

In [9]:
cartpole_env = Environment()
cartpole_env.run()

Net(
  (fc1): Linear(in_features=4, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3_adv): Linear(in_features=32, out_features=2, bias=True)
  (fc3_v): Linear(in_features=32, out_features=1, bias=True)
)
0 Episode: Finished after 30 steps : 10試行の平均step数 = 3.0
1 Episode: Finished after 10 steps : 10試行の平均step数 = 4.0
2 Episode: Finished after 16 steps : 10試行の平均step数 = 5.6
3 Episode: Finished after 17 steps : 10試行の平均step数 = 7.3
4 Episode: Finished after 96 steps : 10試行の平均step数 = 16.9
5 Episode: Finished after 61 steps : 10試行の平均step数 = 23.0
6 Episode: Finished after 75 steps : 10試行の平均step数 = 30.5
7 Episode: Finished after 70 steps : 10試行の平均step数 = 37.5
8 Episode: Finished after 71 steps : 10試行の平均step数 = 44.6
9 Episode: Finished after 51 steps : 10試行の平均step数 = 49.7
10 Episode: Finished after 37 steps : 10試行の平均step数 = 50.4
11 Episode: Finished after 41 steps : 10試行の平均step数 = 53.5
12 Episode: Finished after 75 steps : 10試行の平均step数 = 59.4
13 Episode